Rough implemention of attention model to understand what's going on

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [2]:
tf.reset_default_graph()

In [3]:
source_inputs = [np.random.randint(2,101,n) for n in  np.random.randint(5,20,2500)]
target_inputs = [np.random.randint(2,101,n) for n in  np.random.randint(5,20,2500)]

# print(source_inputs)
# print(target_inputs)

In [4]:
source_lens = list(map(len,source_inputs))
target_lens = list(map(len,target_inputs))

# print(input_lens, output_lens)

In [5]:
padded_inputs = np.array([np.concatenate((i,
            [0 for j in range(20-len(i))])) for i in source_inputs])

In [6]:
source_mask_array = 1.0*(padded_inputs > 0.)
#source_mask_array

In [7]:
padded_outputs = np.array([np.concatenate((i,
            [0 for j in range(20-len(i))])) for i in target_inputs])
#padded_outputs

In [8]:
target_mask_array = 1.0*(padded_outputs>0)
#target_mask_array

In [9]:
batch_size = 3
source_vocab_size = 100
target_vocab_size = 100
embed_size = 200
hidden_size = 250
align_size = 150

In [10]:
source_ids = tf.placeholder(dtype=tf.int32,shape=[None,None])
source_seq_lens = tf.placeholder(dtype=tf.int32,shape=[None])
source_mask = tf.placeholder(dtype=tf.float32,shape=[None,None])

In [11]:
source_embeddings = tf.get_variable('source_embedding_matrix',
                            [source_vocab_size+1, embed_size])
enc_inputs = tf.nn.embedding_lookup(source_embeddings, source_ids)

In [12]:
enc_fw_cell = tf.contrib.rnn.GRUCell(hidden_size)
enc_bw_cell = tf.contrib.rnn.GRUCell(hidden_size)

In [13]:
enc_outputs, states = \
tf.nn.bidirectional_dynamic_rnn(cell_fw = enc_fw_cell, 
                                         cell_bw = enc_bw_cell,
                                         inputs = enc_inputs,
                                         sequence_length = source_seq_lens,
                                         dtype = tf.float32)

In [14]:
concat_enc_outputs = tf.concat(enc_outputs, axis=2)

In [15]:
print(concat_enc_outputs.shape)

(?, ?, 500)


In [16]:
bw_enc_output1 = enc_outputs[-1][:,1,:]

In [17]:
U = tf.get_variable('U',[hidden_size,hidden_size])

In [18]:
decoder_state = tf.matmul(bw_enc_output1,U)

In [19]:
batch_size = tf.placeholder(tf.int32)

In [20]:
W = tf.get_variable('W',[1,2*hidden_size,align_size],dtype=tf.float32)
W_rep = tf.tile(W,[batch_size,1,1])

In [21]:
X = tf.matmul(concat_enc_outputs,W_rep)

In [22]:
print(X.shape)

(?, ?, 150)


In [23]:
V = tf.get_variable('V',[hidden_size,align_size])

In [24]:
v = tf.get_variable('v',[1,align_size,1])

In [25]:
v_rep = tf.tile(v,[batch_size,1,1])

In [26]:
print(v_rep.shape)

(?, 150, 1)


In [283]:
P = tf.get_variable('P',[hidden_size,target_vocab_size+2])
b = tf.get_variable('b',[target_vocab_size+2])

In [284]:
target_embeddings = tf.get_variable('target_embedding_matrix',
                            [target_vocab_size+2, embed_size])

In [285]:
with tf.variable_scope('RNN'):
    decoder_cell = tf.contrib.rnn.GRUCell(hidden_size)

In [286]:
decoder_embed = tf.nn.embedding_lookup(target_embeddings, 
                                       tf.ones([batch_size],tf.int32))

In [287]:
decoder_output = tf.placeholder(tf.int32,[None,None])

In [288]:
decoder_length = 20

In [289]:
target_mask = tf.placeholder(dtype=tf.float32,shape=[None,None])

In [290]:
target_seq_lens = tf.placeholder(dtype=tf.int32,shape=[None])

In [291]:
total_loss = 0

In [292]:
logits = []
for t in range(decoder_length):
    with tf.variable_scope('RNN'):
        if t > 0:
            tf.get_variable_scope().reuse_variables()
        r_t = tf.matmul(decoder_state,V)
        
        tanh_input = X + tf.expand_dims(r_t,axis=1)
        u_t = tf.matmul(tanh_input,v_rep)
        u_t = tf.squeeze(u_t,axis=2)

        exp_u_t = tf.exp(u_t)
        softmax_denom = tf.reduce_sum(exp_u_t*source_mask,axis=1,keep_dims=True)
        a_t = exp_u_t/softmax_denom
        a_t = a_t*source_mask

        a_expn = tf.expand_dims(a_t,axis=1)
        c_t = tf.matmul(a_expn,concat_enc_outputs)
        c_t = tf.squeeze(c_t,axis=1)

        decoder_input = tf.concat([decoder_embed,c_t],
                                  axis=1)
        decoder_state,_ = decoder_cell(decoder_input,
                                       decoder_state) 

        logit = tf.nn.xw_plus_b(decoder_state,P,b)
        logits.append(tf.argmax(logit, 1))
        onehot = tf.one_hot(decoder_output[:,t],
                            depth=target_vocab_size+2)
        
        
        cross_entropy = tf.nn.softmax_cross_entropy_with_logits(
                        logits = logit,
                        labels = onehot
                    )
        
        xe_mask = tf.cast(tf.greater(decoder_output[:,t],0),tf.float32)
        
        cross_entropy = xe_mask*cross_entropy
        
        loss = tf.reduce_sum(cross_entropy)
        
        

        total_loss += loss

In [297]:
logits_mat = tf.stack(logits,axis=0)
    
avg_loss = total_loss / tf.reduce_sum(target_mask)
train_op = tf.train.AdamOptimizer(1e-4).minimize(avg_loss)

In [298]:
logits_mat.shape

TensorShape([Dimension(20), Dimension(None)])

In [299]:
num_examples = len(padded_inputs)

In [300]:
batch_size_ = 100

In [301]:
start = 0

In [302]:
import math

In [335]:
with tf.Session() as sess:
    init_op = tf.global_variables_initializer()
    sess.run(init_op)

    for epoch in range(100):
        print('==Epoch %i=='%epoch)
        for i in range(math.ceil(num_examples/batch_size_)):
            start = i*batch_size_
            end = start + batch_size_
            logits_,avg_loss_,_ = sess.run([logits_mat,avg_loss,train_op],
                                feed_dict={
                                           batch_size:min(num_examples, end) - start,
                                           source_mask:source_mask_array[start:end,:],
                                           target_mask:source_mask_array[start:end,:],
                                           source_ids:padded_inputs[start:end], 
                                           source_seq_lens:source_lens[start:end],
                                           target_seq_lens:source_lens[start:end],
                                           decoder_output:padded_inputs[start:end,:],
                                          })
            
        if epoch%10 == 0:
            choices = np.random.randint(0,batch_size_,5)
            print(logits_.T[choices])
            print(padded_inputs[start:end,:][choices])
        print('==Average Loss=%.5f=='%avg_loss_)
    #     print(logits_,avg_loss_)

==Epoch 0==
[[78 78 95 95 95 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30]
 [ 9  9 95 95 95 95 95 95 95 95 95 30 30 30 30 30 30 30 30 30]
 [95 95 95 95 95 95 30 30 30 30 30 30 30 30 30 30 30 30 30 30]
 [46 46 95 95 95 95 95 95 95 95 95 95 95 95 95 95 95 95 95 95]
 [35 20 44 95 95 95 95 95 95 95 95 95 95 95 95 95 95 95 95 95]]
[[28 23  7 74 95 34  8 12 92 93 67 60 31 91  0  0  0  0  0  0]
 [11 14 61 28 43 20 59 36 80 64 90  3 14 26  0  0  0  0  0  0]
 [89 85 17  2 63 53 84 65 42 22 81 10  0  0  0  0  0  0  0  0]
 [24 52 57 15  5 67 47 40 76 64 37 95 29 91 24  0  0  0  0  0]
 [37 80 77 12  3  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]]
==Average Loss=4.61335==
==Epoch 1==
==Average Loss=4.59202==
==Epoch 2==
==Average Loss=4.56919==
==Epoch 3==
==Average Loss=4.54154==
==Epoch 4==
==Average Loss=4.50101==
==Epoch 5==
==Average Loss=4.42554==
==Epoch 6==
==Average Loss=4.33243==
==Epoch 7==
==Average Loss=4.22893==
==Epoch 8==
==Average Loss=4.13206==
==Epoch 9==
==Average Loss=4.04913=

In [339]:
choices = np.random.randint(0,batch_size_,5)
for choice in choices:
    print(padded_inputs[start:end,:][choice])
    print(logits_.T[choice])
    print()

[49 43 44 60 18 37 34 50 46 25 55 53  0  0  0  0  0  0  0  0]
[43 44 44 60 60 18 34 46 46 55 53 53 53 53 25 25 25 25 25 25]

[84 84  5 17 12 15 28 35  9 68 51 32  0  0  0  0  0  0  0  0]
[84 84  5 17 15 15 15 51 51 32 32 32 32 32 32 32 32 32 32 32]

[74 31 33 87 92 82 69 56 20 67 32 83 33 10 89 60  0  0  0  0]
[31 31 31 87 82 27 23 20 20 20 33 33 33 89 10 10 10 33 33 33]

[24 52 57 15  5 67 47 40 76 64 37 95 29 91 24  0  0  0  0  0]
[17 17 57 15 15 47 47 76 64 95 95 95 29 29 24 24 24 47 76 76]

[71 19 77 40  3 89 31  3 86 78  0  0  0  0  0  0  0  0  0  0]
[19 19 77 40  3  3  3  3 78 78 78 78 31 31 31  3  3  3 78 78]



In [353]:
same = np.sum((logits_.T==padded_inputs[start:end,:]),axis=1)

In [354]:
inds = np.argsort(same)

In [355]:
choices = inds[-10:][::-1]
for choice in choices:
    print(padded_inputs[start:end,:][choice])
    print(logits_.T[choice])
    print(same[choice])
    print()

[ 8 84 26 92 29  4 15 64 33 92 26 25 59 77  7 64 27 55  0  0]
[84 84 26 92 29 15 15 64 64 33 26 26 26 77 64 64 64 64 64 64]
9

[98 69 60 67 40 60 65 13 21 59 85 10 90 48 25  0  0  0  0  0]
[69 69 60 60 40 60 65 59 59 59 10 10 48 48 48 48 48 48 48 48]
8

[ 26  73  52  16  80  61  62  13  45  39  65 100  85  10  10   0   0   0
   0   0]
[ 73  73  16  16  61  61  13  13  39  39  39 100  10  10  10  10  10  39
  39 100]
8

[ 46  54  34  92  38 100  62  62  44  48  27  12  36  49  39  10  91  37
   0   0]
[54 54 34 92 38 62 62 62 12 12 12 12 12 49 49 49 49 49 49 10]
8

[58 61 82 31 64  6 78 51 40 28 32 32 18 12 65 20 93  0  0  0]
[61 61 82 31 64 51 51 32 32 32 32 32 32 12 20 20 20 32 32 32]
8

[ 2 58 31 54 24 94 52 87 80  9 94 72 69 71 98 40  0  0  0  0]
[58 58 31 31 54 54 52 80 80  9 94 72 71 71 71 71 40 98  9  9]
8

[29 86 33 88 95 61 87 65 47  5 69 27  8  3 33 74  0  0  0  0]
[86 33 33 88 64 87 87 65  5  5 69 69 33 33 33 33 47  8  5  5]
7

[11 71 26 61 47 98 74  7 93 24  0  0  0  0  0  0